In [ ]:
%matplotlib inline

# seq2seq
Замысел предложен в работе [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

В чистом виде seq2seq "через" один thought vector не особо работает, поэтому мы ещё добавим механизм внимания.

Он, в свою очередь, был добавлен в эту модель в работе [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473).

Другие полезные статьи:

* [Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
* [Sequence to Sequence Learning with Neural
   Networks](https://arxiv.org/abs/1409.3215)
* [Neural Machine Translation by Jointly Learning to Align and
   Translate](https://arxiv.org/abs/1409.0473)
* [A Neural Conversational Model](https://arxiv.org/abs/1506.05869)


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
random.seed(1234)

Как и во всякой не самой тривиальной постановке задачи, тут будет много вспомогательного кода.

## Загрузка
Занружаем наш датасет


In [ ]:
# prepare data

MAX_LENGTH = 60
df = pd.read_csv('train.tsv', sep='\t')
df

,id,src,tgt
0,0,SPORTSWRITER,S P AO1 R T S R AY2 T ER0
1,1,SKINS,S K IH1 N Z
2,2,ACHEY,AE1 CH IY0
3,3,SPINNER,S P IH1 N ER0
4,4,REALLOCATION,R IY0 AE2 L AH0 K EY1 SH AH0 N
...,...,...,...
65935,65935,MERCS,M ER1 K S
65936,65936,URGE,ER1 JH
65937,65937,MOHAWK,M OW1 HH AO2 K
65938,65938,BALTAZAR,B AA0 L T AA0 Z AA1 R


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('input.txt', 'w') as out_file:
  for x in np.array(df):
    # print(str(x[2]) + '\t' + str(x[1]) + '\n')
    out_file.write(str(x[2]) + '\t' + str(x[1]) + '\n')

one-hot-encod-им каждое слово, при этом обрежем словарь по частотам, чтобы размерность была не километровая.

Как и раньше, пронумеруем слова, и заодно посчитаем количество вхождений -- чтобы потом обрезать словарь по частоте.

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
  
    def __init__(self, name):
        self.name = name
        self.word2index, self.word2count = {}, {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Учитываем SOS и EOS, это счётчик для нумерации

    def add_sentence(self, sentence):

        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):

        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Нормализуем тексты -- приведём к ASCII, приведём в нижний регистр.



In [ ]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    return s

In [ ]:
def read_langs(lang1, lang2, reverse=False):
    print("Reading lines...")

    lines = open('input.txt', encoding='utf-8').\
        read().strip().split('\n')

    # бьём по табу и нормализовываем ТУДУ
    pairs = [[normalizeString(l.split('\t')[0]), ' '.join(l.split('\t')[1].lower().strip())] for l in lines]
    print(pairs[:5])
    # создаём объекты класса Lang
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Теперь соберём все наши функции-заготовки в пайплайн подготовки текстов к подаче модели на вход

In [ ]:
def prepare_data(lang1, lang2, reverse=False):

    input_lang, output_lang, pairs = read_langs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))

    print("Trimmed to %s sentence pairs" % len(pairs))

    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepare_data('wrd', 'phn', True)
print(random.choice(pairs))

Reading lines...
[['s p ao1 r t s r ay2 t er0', 's p o r t s w r i t e r'], ['s k ih1 n z', 's k i n s'], ['ae1 ch iy0', 'a c h e y'], ['s p ih1 n er0', 's p i n n e r'], ['r iy0 ae2 l ah0 k ey1 sh ah0 n', 'r e a l l o c a t i o n']]
Read 65940 sentence pairs
Trimmed to 65940 sentence pairs
Counting words...
Counted words:
phn 30
wrd 72
['p e a r s e', 'p er1 s']


## Собственно модель

В стандартной постановке мы проходим по входной последовательности рекуррентной сетью, и на выходе получаем thought vector, из которого авторегрессивно декодируем последовательность вообще говоря другой длины.




Кодировщик (Encoder)
-----------
Обычная такая RNN-ка, которая использует входную последовательность и предыдущее состояние.



In [ ]:
class EncoderRNN(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Как видите, ничего нового.

Декодировщик (Decoder)
-----------

Берёт вектор(ы) кодировщика и порождает последовательность.




### Декодировщик с механизмом внимания

Смотрим на выходы кодировщика и дополняем их взвешенной обучаемой комбинацией информацию (фичи) на каждом шаге декодировщика (см. ``attn_applied``).

Вычисление весов внимания делается полносвязным слоем ``attn`` которому на вход подаются (а) вход декодера, (б) скрытое состояние декодера. То есть так мы подбираем вес, учитывая и декодируемую последовательность и конкретный "предыдущий токен".

Но при этом нам всё-таки придётся ограничить возможную длину последовательности.





In [ ]:
class AttnDecoderRNN(nn.Module):

    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()

        self.hidden_size, self.output_size = hidden_size, output_size
        self.dropout_p, self.max_length = dropout_p, max_length

        # преобразовываем вход
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)

        # у внимания "двойной" вход, а выход -- 
        # набор скаляров на всю потенциальную длину КОДИРУЕМОГО предложения
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)

        # будем объединять взвешенную сумму выходов кодировщика 
        # и эмбеддинг в один вектор на вход для ячейки GRU 
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)

        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)

        # предсказания
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):

        # построили эмбеддинг
        embedded = self.embedding(input).view(1, 1, -1)
        # применили дропаут
        embedded = self.dropout(embedded)

        # применили линейный слой к преобразованному входу и полученному 
        # скрытому состоянию; получили вектор длины max_length
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        
        # домножили выходы кодировщика на веса внимания
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        # приклеили полученный вектор к эмбеддингу входа
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        # применили нелинейное преобразование
        output = F.relu(output)

        # применили рекуррентную ячейку
        output, hidden = self.gru(output, hidden)

        # применили к выходу софтмакс, чтобы получить очередной токен
        output = F.log_softmax(self.out(output[0]), dim=1)
        
        return output, hidden, attn_weights

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Как мы уже обсудил, это не единственный вариант внимания, можно использовать и более "удачные" в вычислительном плане подходы, например, [Effective Approaches to Attention-based Neural Machine](https://arxiv.org/abs/1508.04025)

## Обучение

Подготовка данных: превращаем тексты в тензоры, не забывая прикреплять SOS/EOS.


In [ ]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensor_from_pair(pair):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

### Собственно обучение

Есть разные подходы, как правильно обучать.

"Teacher forcing" -- подавать на вход декодировщику ПРАВИЛЬНЫЕ, а не предсказанные моделью на этом этапе токены. Сходится быстрее, но когда дело доходит до практики, результаты [очень нестабильные](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

Таким образом обученная модель генерирует часто грамматически корректные предложения, которые имеют мало общего со смыслом исходного текста.




In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, 
          encoder, decoder, 
          encoder_optimizer, decoder_optimizer, # да, два
          criterion, max_length=MAX_LENGTH):
  
    encoder_hidden = encoder.init_hidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0) # первое измерение -- длина
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    # True, если меньше выбранной "доли срабатываний" `teacher_forcing_ratio`
    use_teacher_forcing = random.random() < teacher_forcing_ratio

    if use_teacher_forcing:
        # Teacher forcing: подставляем истинный токен на всей последовательности
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, # индекс
                decoder_hidden, # скрытое состояния
                encoder_outputs # выходы кодировщика
                )
            # сравниваем выход декодировщика с ИСТИННЫМ тензором (это всегда)
            loss += criterion(decoder_output, target_tensor[di])
            # берём в качестве следующего входа ИСТИННЫЙ тензор --
            decoder_input = target_tensor[di]  # ... это teacher forcing

    else:
        # БЕЗ teacher forcing: собственные предсказания как сл. вход
        for di in range(target_length):

            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            # берём самый большой выход софтмакса
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # открепляем от выч. графа
            loss += criterion(decoder_output, target_tensor[di])

            # если предсказали EOS, заканчиваем
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

Вспомогательные функции, чтобы отслеживать, как далеко мы продвинулись и сколько времени прошло.

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

* Запускаем таймер,
* задаём оптимизаторы и функцию потерь,
* создаём набор данных для обучения,
* сохраняем лоссы, чтобы отслеживать, как у нас идут дела, в картинках.

Обучаем на случайных семплах.

In [ ]:
def train_iter(encoder, decoder, 
               print_every=1000, plot_every=100, 
               learning_rate=0.01):
    
    start = time.time()
    plot_losses = []
    print_loss_total, plot_loss_total = 0, 0

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensor_from_pair(pair)
                      for pair in pairs]
    criterion = nn.NLLLoss()

    for iter in range(1, len(training_pairs) + 1):
    # for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, 
                     encoder, decoder, 
                     encoder_optimizer, decoder_optimizer, 
                     criterion)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / len(training_pairs)),
                                         iter, iter / len(training_pairs) * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            # сохраняем средний лосс с прошлого момента измерения
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

### Графики

Строим, как меняется лосс во времени (``plot_losses``) в ходе обучения.




In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    
    # эта штука метки на осях расставляет
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Оценка качества

Примерно то же, что обучение, но, понятно, без teacher forcing. То есть  -- что декодер предсказал на этом шаге, то и подали ему как вход на следующий.

Для анализа впоследствии мы также сохраняем значения внимания.



In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):

    with torch.no_grad():

        input_tensor = tensor_from_sentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.init_hidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden

        # сохраняем для последующего анализа
        decoded_words = [] 
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):

            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)

            # раз            
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                # два
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

Можно просто глазами посмотреть на поведение на отдельных случайных парах

In [ ]:
def evaluate_randomly(encoder, decoder, n=10):
    
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Обучение и оценка

Не забывайте, что датасет игрушечный, на нормальном всё так хорошо работать не будет.


In [ ]:
hidden_size = 128
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

lr = [0.005, 0.003, 0.002, 0.001, 0.001, 0.0009, 0.0008, 0.0007, 0.0006, 0.0005]
epochs = 10
for i in range(epochs):
  print('epoch {}'.format(i + 1))
  train_iter(encoder1, attn_decoder1, print_every=10000, learning_rate = 0.001)

epoch 1
1m 31s (- 8m 29s) (10000 15%) 2.4774
3m 10s (- 7m 16s) (20000 30%) 2.6502
4m 55s (- 5m 53s) (30000 45%) 2.8873
6m 42s (- 4m 21s) (40000 60%) 2.9766
8m 30s (- 2m 42s) (50000 75%) 2.9321
10m 17s (- 1m 1s) (60000 90%) 2.9140
epoch 2
1m 48s (- 10m 6s) (10000 15%) 2.9190
3m 36s (- 8m 16s) (20000 30%) 2.8998
5m 22s (- 6m 26s) (30000 45%) 2.8616
7m 9s (- 4m 38s) (40000 60%) 2.8332
8m 56s (- 2m 51s) (50000 75%) 2.8267
10m 43s (- 1m 3s) (60000 90%) 2.8230
epoch 3
1m 50s (- 10m 17s) (10000 15%) 2.8145
3m 38s (- 8m 21s) (20000 30%) 2.8167
5m 27s (- 6m 32s) (30000 45%) 2.8113
7m 16s (- 4m 43s) (40000 60%) 2.7899
9m 5s (- 2m 53s) (50000 75%) 2.7768
10m 55s (- 1m 4s) (60000 90%) 2.7699
epoch 4
1m 52s (- 10m 30s) (10000 15%) 2.7398
3m 46s (- 8m 39s) (20000 30%) 2.7194
5m 38s (- 6m 46s) (30000 45%) 2.6946
7m 31s (- 4m 52s) (40000 60%) 2.6580
9m 24s (- 2m 59s) (50000 75%) 2.6285
11m 18s (- 1m 7s) (60000 90%) 2.5975
epoch 5
1m 53s (- 10m 36s) (10000 15%) 2.5445
3m 46s (- 8m 41s) (20000 30%) 2.50

In [ ]:
evaluate_randomly(encoder1, attn_decoder1)

# Вывод

In [1]:
def write_solution(encoder, decoder):
  outs = []
  df = pd.read_csv('test2.tsv', sep='\t')
  out_df = pd.DataFrame(columns=['id', 'tgt']).reset_index(drop=True)
  for i in np.array(df):
    word = ' '.join(i[1]).lower().strip()
    output_words, attentions = evaluate(encoder, decoder, word)
    output_sentence = ' '.join(output_words)[:-6]
    out_df = out_df.append({'id': i[0], 'tgt': output_sentence.upper().strip()}, ignore_index=True)
  out_df = out_df.set_index('id')
  out_df.to_csv('submission.csv')
  return out_df

In [ ]:
out = write_solution(encoder1, attn_decoder1)
out.head()